In [ ]:
import numpy as np 
import cv2 


source = cv2.imread("in_11.png") 
blend = cv2.imread("tar_11.png")
height, width, channel = source.shape

mask = np.zeros((700,530,3), dtype='float32')
mask[250:400,140:300,:] = (1,1,1)



#masked_img = cv2.bitwise_and(blend,blend,mask = mask)
 

# generating Gaussian pyramid for Source
G1 = source.copy()
gpS = [G1]
for i in range(5):
    G1 = cv2.pyrDown(G1)
    gpS.append(G1)
# generating Gaussian pyramid for Blend

G2 = blend.copy()
gpM = [G2]
for i in range(5):
    G2 = cv2.pyrDown(G2)
    gpM.append(G2)
    
# generating Gaussian pyramid for Mask    
M = mask.copy()
gpR = [M]
for i in range(5):
    M = cv2.pyrDown(M)
    gpR.append(M)
        
   
    
# generate Laplacian Pyramid for Source
lpTopS = gpS[-1]
lpS = [lpTopS]
for i in range(5,0,-1):
    sizeS = gpS[i-1].shape[1], gpS[i-1].shape[0]
    GE1 = cv2.pyrUp(gpS[i], dstsize=sizeS)

    L = cv2.subtract(gpS[i-1], GE1)
    lpS.append(L)
    
# generate Laplacian Pyramid for Blend
lpTopM = gpM[-1]
lpM = [lpTopM]
for i in range(5,0,-1):
    sizeM = gpM[i-1].shape[1], gpM[i-1].shape[0]
    GE2 = cv2.pyrUp(gpM[i], dstsize=sizeM)
    L = cv2.subtract(gpM[i-1], GE2)
    lpM.append(L)

def blendimage(A, B, mask_pyr):
    LS = []
    for la, lb, msk in zip(A,B,mask_pyr):
        ls = la*msk + lb*(1.0-msk)
        LS.append(ls)
    return LS

def turnoriginal(laplacian):
    lp_top = laplacian[0]
    lp_lst =  [lp_top]
    num_levels = len(laplacian) - 1
    for i in range(num_levels):
        size = (laplacian[i+1].shape[1], laplacian[i+1].shape[0])
        laplacian_e = cv2.pyrUp(lp_top, dstsize=size)
        lp_top = cv2.add(laplacian[i+1], laplacian_e)
        lp_lst.append(lp_top)

    return lp_lst
gpR.reverse()
added_lp = blendimage(lpS,lpM, gpR)



result = turnoriginal(added_lp)



cv2.imshow("Image", added_lp[5])
cv2.waitKey(0)